<a href="https://colab.research.google.com/github/Eliezer19Garcia/Machine_Learning/blob/main/Metrics_evaluate_clustering(ML_S11).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

np.random.seed(42)

In [ ]:
from sklearn.datasets import fetch_20newsgroups_vectorized, fetch_20newsgroups

In [ ]:
news_20 = fetch_20newsgroups()


In [ ]:
news_20.data[:2]
fetch_20newsgroups_vectorized?
datos = fetch_20newsgroups_vectorized()
noticias = datos.data

In [ ]:
noticias.shape

(11314, 130107)

In [ ]:

print(len(datos.target_names))
datos.target_names

20


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [ ]:
from sklearn.cluster import MiniBatchKMeans

In [ ]:
estimador = MiniBatchKMeans(n_clusters=20)
estimador.fit(noticias)
etiquetas_pred = estimador.labels_

In [ ]:
etiquetas_pred

array([ 9,  8, 12, ..., 16,  1, 12], dtype=int32)

### Medidas de evaluación externa
Son aquellas en las que las etiquetas de clusters son conocidas de antemano.

In [ ]:
clases = datos.target


In [ ]:
from sklearn.metrics import homogeneity_completeness_v_measure, adjusted_rand_score

La función homogeneity_completeness_v_measure nos devuelve una tupla con la homogeneidad, la completación y la medida v

In [ ]:
homogeneity_completeness_v_measure(clases, etiquetas_pred) #homogeneity , completeness , and vmeasure(has to be near to 1 to be a good model)

(np.float64(0.07888488977731375),
 np.float64(0.09053375607377358),
 np.float64(0.08430884727154726))

Vemos que estos clusters tienen mas completación que homogeneidad (es decir, los clusters son menos homogéneos que la homogeneidad de la distribución de las clases en clusters.

El v-measure tiene un rango de (0, 1) con lo cual estos clusters no son muy buenos.

Un problema que tiene el criterio de homogeneidad y completación es que no son invariantes del orden de las clases.

Para datasets de menos de 1000 observaciones o para un número de clusters mayor que 10, se recomienda usar el Indice de Rand ajustado, adjusted_rand_score.

In [ ]:
adjusted_rand_score(clases, etiquetas_pred)

0.028039527074345708

La métrica ARI tiene un rango de (-1,1) asi que pese a que este grupo de clusters no es el peor, no es muy bueno

Podemos usar medidas de evaluación externa para hacer validación cruzada como haríamos con cualquier problema de regresión/clasificación.

Métricas definidas en cross_val_score incluyen:

-   adjusted_rand_score
-   completeness_score
-   homogeneity_score
-   v_measure_score

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
resultados = cross_val_score(estimador, noticias, clases, scoring='adjusted_rand_score', cv=5)

In [ ]:
resultados.mean()

np.float64(0.026674084479600714)

##Medidas de evaluación interna
Dichas medidas se usan cuando las clases verdaderas no se conocen de antemano

In [ ]:
from sklearn.metrics import silhouette_score

Estas medidas internas se usan en sklearn pasando dos argumentos, el dataset de entrenamiento y las etiquetas de los clusters.

En primer lugar tenemos el Coeficiente de Silueta (silhouette_score)

In [ ]:
silhouette_score(noticias, etiquetas_pred) # The conclusion is that isnt a good idea take 20 clusters

np.float64(-0.04858106380375954)

In [ ]:
for k in [5,10,20,30]:
    estimador = MiniBatchKMeans(n_clusters=k)
    estimador.fit(noticias)
    etiquetas_pred = estimador.labels_
    print(f"{(k,silhouette_score(noticias, etiquetas_pred))}")

(5, np.float64(-0.010166656407493965))
(10, np.float64(-0.04918373746067698))
(20, np.float64(-0.0686532661525218))
(30, np.float64(-0.09293934154974755))
